In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# meow meow meow meow meow! meow mewoe moew meowm eow !hacker debug meow debug debug meow hackechakcher hckahcer !meiw!

Resize and greyscale images:

In [2]:
image_directory = '../images/dataset'
starters = ["Bulbasaur", "Charmander", "Squirtle"]

image_data = []
labels = []

pokemon_to_feature_num = {
    "Bulbasaur": 0,
    "Charmander": 1,
    "Squirtle": 2,

}

for starter in starters:
    starter_folder = os.path.join(image_directory, starter)
    
    # Make sure the folder exists
    if os.path.isdir(starter_folder):
        for filename in os.listdir(starter_folder):
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith('.jfif'):
                image_path = os.path.join(starter_folder, filename)
                
                # Open the image and resize it to a fixed size 
                img = Image.open(image_path).resize((128, 128))
                #Greyscale
                # img = img.convert('L')
                # Convert to RGB if it's RGBA
                if img.mode == 'RGBA':
                    img = img.convert('RGB')  # Discard the alpha channel
                img_array = np.array(img)
                
                if img_array.shape != (128, 128, 3):
                    print(f"Skipping {filename}, invalid shape: {img_array.shape}")
                    continue
                else:
                    image_data.append(img_array)
                    labels.append(pokemon_to_feature_num[starter])

labels = np.array(labels)
image_data = np.array(image_data)

Skipping 00000114.png, invalid shape: (128, 128)
Skipping 00000190.png, invalid shape: (128, 128)
Skipping 00000172.png, invalid shape: (128, 128)
Skipping 00000061.png, invalid shape: (128, 128)
Skipping 59bef3942b6041b3a6e0526100264536.jpg, invalid shape: (128, 128)
Skipping 00000078.png, invalid shape: (128, 128)
Skipping 00000009.png, invalid shape: (128, 128)
Skipping 00000021.png, invalid shape: (128, 128)


Display the image in greyscale to check data

In [28]:
img = image_data[0].reshape(128, 128,3)
img_pillow = Image.fromarray(img)
img_pillow.show()

In [4]:
print(image_data.shape)
print(labels.shape)

(847, 128, 128, 3)
(847,)


Split data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.3, random_state=420)

Model:

In [10]:
import tensorflow as tf

from tensorflow.keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization()) # does something good, maybe i'll read more about it but i just added it and it helped a lot
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())#1d for neural network

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.45)) #address overfitting
model.add(layers.Dense(3, activation='softmax'))

In [11]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 859,523 (3.28 MB)

 Trainable params: 859,459 (3.28 MB)

 Non-trainable params: 64 (256.00 B)

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])


In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('./models/best_model1.keras', 
                             monitor='val_accuracy',    # Monitor validation accuracy
                             save_best_only=True,       # Save only the best weights
                             mode='max',                # 'max' means we want to maximize the metric
                             verbose=1)

history = model.fit(X_train, y_train, 
                    epochs=75, 
                    batch_size=32, 
                    validation_data=(X_test, y_test),
                    callbacks= [checkpoint])

Epoch 1/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.4451 - loss: 2.3348
Epoch 1: val_accuracy improved from -inf to 0.92157, saving model to ./models/best_model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 433ms/step - accuracy: 0.4524 - loss: 2.2889 - val_accuracy: 0.9216 - val_loss: 0.3772
Epoch 2/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - accuracy: 0.8514 - loss: 0.4348
Epoch 2: val_accuracy improved from 0.92157 to 0.95294, saving model to ./models/best_model1.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 401ms/step - accuracy: 0.8530 - loss: 0.4308 - val_accuracy: 0.9529 - val_loss: 0.1216
Epoch 3/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.9201 - loss: 0.2530
Epoch 3: val_accuracy did not improve from 0.95294
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 414ms/step - accuracy: 0.9208 - loss: 0.2509 - val_accuracy: 0.8980 - val_loss: 0.2958
Epoch 4/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - accuracy: 0.9635 - loss: 0.1419
Epoch 4: val_accuracy improved from 0.95294 to 0.97647, sav

test on hand drawn images

In [15]:
best_model1 = tf.keras.models.load_model('./models/best_model1.keras')
best_model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       802,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,578,443 (9.84 MB)

 Trainable params: 859,459 (3.28 MB)

 Non-trainable params: 64 (256.00 B)

 Optimizer params: 1,718,920 (6.56 MB)

In [16]:
hand_image_directory = '../images/Hand_Drawn'

In [17]:

hand_data = []
if os.path.isdir(hand_image_directory):
    for filename in os.listdir(hand_image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith('.jfif'):
            image_path = os.path.join(hand_image_directory, filename)
            
            # Open the image and resize it to a fixed size 
            img = Image.open(image_path).resize((128, 128))
            #Greyscale
            # img = img.convert('L')
            # Convert to RGB if it's RGBA
            if img.mode == 'RGBA':
                img = img.convert('RGB')  # Discard the alpha channel
            img_array = np.array(img)
            
            if img_array.shape != (128, 128, 3):
                print(f"Skipping {filename}, invalid shape: {img_array.shape}")
                continue
            else:
                hand_data.append(img_array)

hand_data = np.array(hand_data)

In [18]:
img = hand_data[0].reshape(128,128 ,3)
img_pillow = Image.fromarray(img)
img_pillow.show()

In [33]:
pred_probs = best_model1.predict(hand_data)
print(pred_probs)
predicted_class = np.argmax(pred_probs, axis=1)
print(predicted_class) # i manually compared by looking in folder

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
[[2.3090009e-01 4.4711931e-03 7.6462865e-01]
 [4.1295946e-02 3.6413595e-03 9.5506269e-01]
 [4.0975580e-04 3.1825132e-07 9.9958998e-01]
 [1.0722476e-04 9.9885875e-01 1.0340671e-03]
 [7.5347386e-02 9.4431022e-04 9.2370832e-01]
 [2.2635427e-01 2.4640688e-01 5.2723891e-01]
 [5.9589869e-01 1.3361390e-03 4.0276518e-01]]
[2 2 2 1 2 2 0]


Note: 30 epochs for all the colored ones to be guessed correctly..., non colored defaults to squirtle

Ok lets see what happens with transformations on the data.

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#datagen is interesting
datagen = ImageDataGenerator(
    rotation_range=30,              # Randomly rotate images by up to x degrees
    width_shift_range=0.2,          # Randomly shift images horizontally by x%
    height_shift_range=0.2,         # Randomly shift images vertically by x%
    shear_range=0.2,                # Apply shear transformations
    zoom_range=0.2,                 # Random zoom
    horizontal_flip=True,           # Randomly flip images horizontally
    fill_mode='nearest'             # Strategy for filling in missing pixels (due to rotation or shift)
)

datagen.fit(X_train)

In [21]:
import tensorflow as tf

from tensorflow.keras import layers, models

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model2.add(layers.BatchNormalization())
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())

model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dropout(0.45)) #address overfitting
model2.add(layers.Dense(3, activation='softmax'))

model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])



/Users/tylercako/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('./models/best_model.keras', 
                             monitor='val_accuracy',    # Monitor validation accuracy
                             save_best_only=True,       # Save only the best weights
                             mode='max',                # 'max' means we want to maximize the metric
                             verbose=1)

history = model2.fit(
    datagen.flow(X_train, y_train, batch_size=32),  # Augmented images in batches
    epochs=75,
    validation_data=(X_test, y_test),
    callbacks=[checkpoint]
)

/Users/tylercako/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step - accuracy: 0.4845 - loss: 2.0670
Epoch 1: val_accuracy improved from -inf to 0.68627, saving model to ./models/best_model.keras
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 536ms/step - accuracy: 0.4902 - loss: 2.0291 - val_accuracy: 0.6863 - val_loss: 0.7469
Epoch 2/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step - accuracy: 0.9559 - loss: 0.1362
Epoch 31: val_accuracy did not improve from 0.98824
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 492ms/step - accuracy: 0.9558 - loss: 0.1362 - val_accuracy: 0.9569 - val_loss: 0.2338
Epoch 32/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step - accuracy: 0.9651 - loss: 0.1592
Epoch 32: val_accuracy did not improve from 0.98824
19/19 ━━━━━━━━━━━━━━━━━━━━ 10s 496ms/step - accuracy: 0.9652 - loss: 0.1583 - val_accuracy: 0.9569 - val_loss: 0.2322
Epoch 33/75
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step - accuracy: 0.9542 - loss: 0.1823
Epoch 33: val_accuracy did not improve from 0.98824
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 457ms/step - accurac

In [23]:
best_model = tf.keras.models.load_model('./models/best_model.keras')
best_model.evaluate(X_test, y_test)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 1.0000 - loss: 0.0100


[0.012258642353117466, 1.0]

In [ ]:
#Export model to teraform.js file
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(best_model, "./models/best_model.)

In [26]:
pred_probs = best_model.predict(hand_data)
print(pred_probs)
predicted_class = np.argmax(pred_probs, axis=1)
print(predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
[[5.4474878e-01 1.5566649e-01 2.9958472e-01]
 [4.3693358e-01 1.8039298e-01 3.8267341e-01]
 [6.8670365e-06 4.3074175e-09 9.9999309e-01]
 [4.3686599e-12 1.0000000e+00 4.5571175e-13]
 [9.9991298e-01 2.2893381e-08 8.6993401e-05]
 [7.3719418e-01 1.0802829e-01 1.5477748e-01]
 [4.6723655e-01 1.5960170e-01 3.7316170e-01]]
[0 0 2 1 0 0 0]


In [27]:
pred_probs = model2.predict(hand_data)
print(pred_probs)
predicted_class = np.argmax(pred_probs, axis=1)
print(predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
[[5.9174842e-01 1.7366808e-02 3.9088479e-01]
 [4.8120430e-01 1.7500995e-02 5.0129467e-01]
 [2.5736790e-07 3.0621537e-13 9.9999976e-01]
 [4.9173009e-12 1.0000000e+00 2.9472631e-12]
 [9.9863064e-01 8.6849646e-08 1.3692196e-03]
 [8.8014203e-01 4.1595478e-02 7.8262582e-02]
 [5.3414977e-01 1.4648564e-02 4.5120174e-01]]
[0 2 2 1 0 0 0]
